In [1]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import joblib 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import fbeta_score

df= pd.read_csv('../data/GeneralDatensatz18-21ohneGeo-mitLockdown_mitCorona_mitFerien.csv', sep=';')



X_train_val = df.drop(columns=['UKATEGORIE','LAND','LOR_ab_2021'])
y_train_val = df['UKATEGORIE'].isin([1, 2]).astype(int)

# Stratified K-Fold Cross Validation Setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_val_fbeta_scores = []  # Liste, um die F-beta Scores der einzelnen Folds zu speichern

for train_index, val_index in skf.split(X_train_val, y_train_val):
    X_train, X_val = X_train_val.iloc[train_index], X_train_val.iloc[val_index]
    y_train, y_val = y_train_val.iloc[train_index], y_train_val.iloc[val_index]

    # SMOTE anwenden auf den Trainingsdaten
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Trainiere das KNN Modell
    model = KNeighborsClassifier(n_neighbors=6, leaf_size=41, weights='distance', p=1)
    model.fit(X_train_resampled, y_train_resampled)

    # Vorhersagen auf den Validierungsdaten
    y_val_pred = model.predict(X_val)

    # Berechne den F-beta Score für die Validierungsdaten
    val_fbeta = fbeta_score(y_val, y_val_pred, beta=2, average='weighted')
    train_val_fbeta_scores.append(val_fbeta)

# Durchschnittlicher F-beta Score aus der Stratified K-Fold Cross Validation
average_train_val_fbeta = sum(train_val_fbeta_scores) / len(train_val_fbeta_scores)

# Lade den separaten Testdatensatz
test_data = pd.read_csv('../data/test_data_nichtAnfassen.csv')
X_test = test_data.drop(columns=['UKATEGORIE'])
y_test = test_data['UKATEGORIE']

# Trainiere das Modell auf dem gesamten Trainings- und Validierungsdatensatz mit SMOTE
X_train_val_resampled, y_train_val_resampled = smote.fit_resample(X_train_val, y_train_val)
model.fit(X_train_val_resampled, y_train_val_resampled)

# Speichern des Modells
#joblib.dump(model, 'knn_model.pkl')

['knn_model.pkl']